In [1]:
# Notebook til at forsøge at fremskrive vaccinationsdata
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_vacc"
rootdir = os.getcwd() +"\\" + ssidatapath

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])

print(latestdir)
print(latestDate)

filename = 'Vaccinerede_pr_uge_pr_maalgruppe.csv'
curPath = latestdir+"/Vaccine_maalgrupper_DB/" + filename

df_weekly = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')


filename = 'PaabegVacc_daek_DK_prdag.csv'
# curPath = "D:/Pandemix/Github/DanskeData/ssi_vacc/SSI_vacc_2021-05-25/Vaccine_DB/" + filename
curPath = latestdir+"/Vaccine_DB/" + filename


# df = pd.read_csv(curPath,delimiter = ',',dtype=str,encoding='latin-1')
df = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')


filename = 'FaerdigVacc_daekning_DK_prdag.csv'
curPath = latestdir+"/Vaccine_DB/" + filename

df_full = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')


c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-08-02
2021-08-02 00:00:00


In [3]:
datesOnce = pd.to_datetime(df.Vaccinedato)
datesFull = pd.to_datetime(df_full.Vaccinedato)
vaccOnce = pd.to_numeric(df['Kumuleret antal påbegyndt vacc.'])
vaccFull = pd.to_numeric(df_full['Kumuleret antal færdigvacc.'])


fig,ax1 = plt.subplots()

ax1.plot(datesOnce,vaccOnce)
ax1.plot(datesFull,vaccFull)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:

# Drop the three extra empty rows are the top
df_weekly.drop([0,1,2],axis=0,inplace=True)

allDates = df_weekly.Ugenummer

curWeekDays = df_weekly.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")))

allVaccOnce = pd.to_numeric(df_weekly['Dækning førstegangsvaccinerede (kumulativt i %)'])
allVaccFull = pd.to_numeric(df_weekly['Dækning færdigvaccinerede (kumulativt i %)'])
allGroups = df_weekly['Målgruppe'].unique()

# Move groups around to have correct order
allGroups.sort()
allGroups = np.roll(allGroups,-8)
allGroups = np.insert(allGroups,0,allGroups[-8])
allGroups = np.delete(allGroups,-8)
allGroups = np.append(allGroups,allGroups[-8])
allGroups = np.delete(allGroups,-9)
allGroups


array(['1. Plejehjemsbeboere',
       '2. Borgere > 65, praktisk hjælp og personlig pleje',
       '3. Borgere fra årgang 1936 og derunder (85 år og ældre)',
       '4. Personale i sundhedsvæsenet og dele af socialvæsenet',
       '5. Udvalgte patienter med særligt øget risiko',
       '6. Udvalgte pårørende til personer med særligt øget risiko',
       '7. Personer fra årgang 1937-1941',
       '8. Personer fra årgang 1942-1946',
       '9. Personer fra årgang 1947-1956',
       '10A. Personer fra årgang 1957-1961',
       '10B. Personer fra årgang 1962-1966',
       '10C. Personer fra årgang 1967-1971',
       '10D1. Personer fra årgang 1972-1976 og årgang 2002-2005',
       '10D2. Personer fra årgang 1977-1981 og årgang 1997-2001',
       '10D3. Personer fra årgang 1982-1986 og årgang 1992-1996',
       '10D4. Personer fra årgang 1987-1991', 'Resterende'], dtype=object)

In [5]:
# Hard-coded fix of the missing data-points
group5Len = len(df_weekly[df_weekly['Målgruppe'] == allGroups[5]])
otherGroupLen = len(df_weekly[df_weekly['Målgruppe'] == allGroups[2]])
if (group5Len == otherGroupLen - 3):

    rowToCopy = df_weekly.loc[57]
    toAdd1 = rowToCopy.copy()
    toAdd2 = rowToCopy.copy()
    toAdd3 = rowToCopy.copy()
    toAdd1.Ugenummer = '2021-W03'
    toAdd2.Ugenummer = '2021-W04'
    toAdd3.Ugenummer = '2021-W06'

    df_weekly = df_weekly.append(toAdd1)
    df_weekly = df_weekly.append(toAdd2)
    df_weekly = df_weekly.append(toAdd3)

# Sort so order is correct
df_weekly.sort_values(by=['Ugenummer','Målgruppe'],inplace=True)

In [6]:
def logiGrowth(x,K,r,x0):
    
    curBot = 1 + ((K-x0)/x0) * np.exp(-r*x)
    
    return K/curBot

In [7]:
xRange = np.arange(0,20,0.1)

# testPars = (0.9,0.5,0.01)

testY = logiGrowth(xRange,0.9,0.5,0.01)

fig,ax1 = plt.subplots()

ax1.plot(xRange,testY)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:


# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots()


for k in range(len(allGroups)-5,len(allGroups)):
# for k in range(0,len(allGroups)):
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]
    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))).values

    ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)']).values

    datToUse = 5

    xPlot = ageWeekDays[datToUse:]
    yPlot = ageVaccOnce[datToUse:]
    xVals = ageWeekDays[datToUse:]
    xVals = np.array(xVals,dtype='datetime64[D]')
    xVals = xVals-xVals[0]
    xVals = xVals/np.timedelta64(1,'D')

    from scipy.optimize import curve_fit
    popt, pcov = curve_fit(logiGrowth,xVals,yPlot)


    ax1.plot(xPlot,logiGrowth(xVals,*popt),'k')
    ax1.plot(xPlot,yPlot,'*',label=curLabel+f' Max: {popt[0]:2.0f}')

ax1.set_ylim([0,100])
ax1.legend(fontsize=10)

ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-6-acd433c12699>:3: RuntimeWarning: overflow encountered in exp
  curBot = 1 + ((K-x0)/x0) * np.exp(-r*x)


In [34]:
# Try an alternative function
def newFunc(x,K,r,p0):
    return  p0* np.exp(-r*x) + K
    # return K*(1-np.exp(-r*x))

fig,ax1 = plt.subplots()

xRange = np.arange(0,37)

ax1.plot(xRange,newFunc(xRange,30,0.1,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

22

In [170]:


# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots()

cmap = cm.get_cmap('twilight',len(allGroups))

# for k in range(len(allGroups)-8,len(allGroups)-3):
for k in range(len(allGroups)-12,len(allGroups)-2):
# for k in range(0,len(allGroups)):

    datToUse = -12
    
    
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]
    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w"))).values

    ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)']).values
    
    firstValsToUse = 45
    firstIndexToUse = np.where(ageVaccOnce > firstValsToUse)[0][0]
    datToUse = firstIndexToUse
    # datToUse = -8
    xPlot = ageWeekDays[datToUse:]
    yPlot = ageVaccOnce[datToUse:]
    
    xVals = xPlot
    xVals = np.array(xVals,dtype='datetime64[D]')
    xVals = xVals-xVals[0]
    xVals = xVals/np.timedelta64(1,'D')
    
    yFit = 100-yPlot

    from scipy.optimize import curve_fit
    # popt, pcov = curve_fit(newFunc,xVals,yFit)
    p0 = [20,0.06,70]
    popt, pcov = curve_fit(newFunc,xVals,yFit,p0=p0)


    ax1.plot(xPlot,100-newFunc(xVals,*popt),color=cmap(k))
    ax1.plot(xPlot,100-yFit,'*',label=curLabel+f', Max: {100-popt[0]:2.0f}',color=cmap(k))
    ax1.plot(ageWeekDays[:datToUse+1],ageVaccOnce[:datToUse+1],':.',color=cmap(k))
    
    xPred = np.arange(xPlot[0] ,xPlot[0] + np.timedelta64(120,'D'),np.timedelta64(7,'D'))
    xPredMod = (xPred-xPred[0]) / np.timedelta64(1,'D')
    
    ax1.plot(xPred,100-newFunc(xPredMod,*popt),'--',color=cmap(k))
    
    

ax1.set_ylim([0,100])
ax1.legend(fontsize=10)

ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figsVacc/VaccinationFremskrivning')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\rakrpe\Anaconda3\envs\main\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [92]:
# Load nøgletal
filename = 'Noegletal_vacc_daekning.csv'
curPath = latestdir+"/Vaccine_maalgrupper_DB/" + filename

df_noegle = pd.read_csv(curPath,delimiter = ';',dtype=str,encoding='latin-1')

In [158]:
# print(allGroups)
# print(len(allGroups))
hardcodedMaxVacc = 96*np.ones((17,))

# hardcodedMaxVacc[ 9] = 92
# hardcodedMaxVacc[10] = 91
# hardcodedMaxVacc[11] = 89
# hardcodedMaxVacc[12] = 85
# hardcodedMaxVacc[13] = 81
# hardcodedMaxVacc[14] = 74
# hardcodedMaxVacc[15] = 74
# hardcodedMaxVacc[16] = 5

hardcodedMaxVacc[ 9] = 92
hardcodedMaxVacc[10] = 91
hardcodedMaxVacc[11] = 89
hardcodedMaxVacc[12] = 85
hardcodedMaxVacc[13] = 81
hardcodedMaxVacc[14] = 80
hardcodedMaxVacc[15] = 80
hardcodedMaxVacc[16] = 5


np.stack([allGroups, hardcodedMaxVacc],axis=1)

array([['1. Plejehjemsbeboere', 96.0],
       ['2. Borgere > 65, praktisk hjælp og personlig pleje', 96.0],
       ['3. Borgere fra årgang 1936 og derunder (85 år og ældre)', 96.0],
       ['4. Personale i sundhedsvæsenet og dele af socialvæsenet', 96.0],
       ['5. Udvalgte patienter med særligt øget risiko', 96.0],
       ['6. Udvalgte pårørende til personer med særligt øget risiko',
        96.0],
       ['7. Personer fra årgang 1937-1941', 96.0],
       ['8. Personer fra årgang 1942-1946', 96.0],
       ['9. Personer fra årgang 1947-1956', 96.0],
       ['10A. Personer fra årgang 1957-1961', 92.0],
       ['10B. Personer fra årgang 1962-1966', 91.0],
       ['10C. Personer fra årgang 1967-1971', 89.0],
       ['10D1. Personer fra årgang 1972-1976 og årgang 2002-2005', 85.0],
       ['10D2. Personer fra årgang 1977-1981 og årgang 1997-2001', 81.0],
       ['10D3. Personer fra årgang 1982-1986 og årgang 1992-1996', 80.0],
       ['10D4. Personer fra årgang 1987-1991', 80.0],
       

In [145]:
# df_noegle
# Antal førstevaccinerede
# Antal færdigvaccinerede 

In [182]:
numNotVaccedPred = []
numVaccedPred =[]

realOnce = []
realFull = []
allPop = []
for k in np.arange(len(allGroups)):
    curGroup = allGroups[k]
    thisdf = df_noegle[df_noegle['Målgruppe'] == curGroup]
    curPop = thisdf.Population.astype(int).sum()
    
    allPop.append(curPop)
    
    curOnce = thisdf['Antal førstevaccinerede'].astype(int).sum()
    curFull = thisdf['Antal færdigvaccinerede'].astype(int).sum()
    
    
    realOnce.append(curOnce)
    realFull.append(curFull)
    
    curNotVaccedPred = curPop * ((100 -  hardcodedMaxVacc[k])/100)
    curVaccedPred = curPop * hardcodedMaxVacc[k]/100
    numNotVaccedPred.append(curNotVaccedPred)
    numVaccedPred.append(curVaccedPred)
# curPop
# numNotVaccedPred

fig,ax1 = plt.subplots(figsize=(12,14))

totUnVaccPred = np.sum(numNotVaccedPred)
totVaccPred = np.sum(numVaccedPred)
totOnceReal = np.sum(realOnce)
totFullReal = np.sum(realFull)

# ax1.bar(allGroups,numNotVaccedPred,color='k',label=f'Ikke vaccineret: {totUnVaccPred:8.0f}')
# ax1.bar(allGroups,numVaccedPred,bottom=numNotVaccedPred,color='green',label=f'Vaccineret: {totVaccPred:15.0f}')

ax1.bar(allGroups,allPop,color='k',label=f'Uvaccineret (fremskrivning): {totUnVaccPred:8.0f}')
ax1.bar(allGroups,numVaccedPred,color='m',label=f'Vaccineret (fremskrivning): {totVaccPred:8.0f}')
ax1.bar(allGroups,realOnce,color='xkcd:light green',label=f'Første dosis: {totOnceReal:8.0f}')
ax1.bar(allGroups,realFull,color='g',label=f'Fuldt vaccineret: {totFullReal:8.0f}')

ax1.legend()

ax1.set_xticklabels(allGroups, rotation = 90, fontsize=12)
plt.tight_layout()

# print(totVaccPred/(totUnVaccPred+totVaccPred))
# print(totVaccPred/np.sum(allPop))
# print(np.sum(realOnce)/np.sum(allPop))

percVaccReal = np.sum(realOnce)/np.sum(allPop)
percVaccPred = totVaccPred/np.sum(allPop)

textToShow = f'Faktiske vaccinerede: {100*percVaccReal:2.1f}%\nFremskrivning: {100*percVaccPred:2.1f}%'
ax1.text(allGroups[7],7.5*1e5,textToShow,fontsize=24,ha='right')

if saveFigures:
    plt.savefig('figsVacc/VaccinationFremskrivningBar')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-182-dd8e47194def>:45: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(allGroups, rotation = 90, fontsize=12)


In [142]:
df_noegle

,Målgruppe,Ansættelsessted,Bopælsregion,Population,Antal førstevaccinerede,Antal færdigvaccinerede,Førstegangs vaccinerede dækning (%),Færdigvaccinerede dækning (%)
0,1. Plejehjemsbeboere,NaN,Hovedstaden,12610,12134,11883,96.2,94.2
1,1. Plejehjemsbeboere,NaN,Midtjylland,9660,9336,9203,96.6,95.3
2,1. Plejehjemsbeboere,NaN,Nordjylland,5332,5138,5034,96.4,94.4
3,1. Plejehjemsbeboere,NaN,Sjælland,5701,5453,5350,95.6,93.8
4,1. Plejehjemsbeboere,NaN,Syddanmark,9375,8998,8800,96.0,93.9
5,"2. Borgere > 65, praktisk hjælp og personlig p...",NaN,Hovedstaden,15622,14339,14087,91.8,90.2
6,"2. Borgere > 65, praktisk hjælp og personlig p...",NaN,Midtjylland,9928,9363,9250,94.3,93.2
7,"2. Borgere > 65, praktisk hjælp og personlig p...",NaN,Nordjylland,5568,5234,5147,94.0,92.4
8,"2. Borgere > 65, praktisk hjælp og personlig p...",NaN,Sjælland,9492,8899,8756,93.8,92.2
9,"2. Borgere > 65, praktisk hjælp og personlig p...",NaN,Syddanmark,13355,12516,12338,93.7,92.4


In [11]:
# Try to plot difference from week to week

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots()


# for k in range(len(allGroups)-5,len(allGroups)):
for k in range(0,len(allGroups)):
    curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

    curLabel = allGroups[k]

    # curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[8]]
    # curLabel = allGroups[8]

    ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")))

    ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)']).values
    ageVaccFull = pd.to_numeric(curdf['Dækning færdigvaccinerede (kumulativt i %)']).values

    # ax1.plot(ageWeekDays,ageVaccOnce,'-*',label=curLabel)
    ax1.plot(ageWeekDays[1:],np.diff(ageVaccOnce),'-*',label=curLabel)

# ax1.set_ylim([0,100])
ax1.legend(fontsize=10)

ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[0]]

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# for k in range(len(allGroups)-5,len(allGroups)-1):
#     curdf = df_weekly[df_weekly['Målgruppe'] == allGroups[k]]

#     curLabel = allGroups[k]

#     ageWeekDays = curdf.Ugenummer.apply(lambda x: np.datetime64(datetime.datetime.strptime(x + '-1', "%Y-W%W-%w")))

#     ageVaccOnce = pd.to_numeric(curdf['Dækning førstegangsvaccinerede (kumulativt i %)'])
#     ageVaccFull = pd.to_numeric(curdf['Dækning færdigvaccinerede (kumulativt i %)'])
    
#     ax1.plot(ageWeekDays,ageVaccOnce,'-*',label=curLabel)
#     ax2.plot(ageWeekDays,ageVaccFull,'-*',label=curLabel)

# ax1.set_ylim([0,100])
# ax2.set_ylim([0,100])
# ax1.legend(fontsize=10)

# ax1.set_xlim(left=np.datetime64('2021-05-01'))
# plt.tight_layout()